<a href="https://colab.research.google.com/github/Vatsaly76/text-to-image/blob/main/Image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate gradio -q

from diffusers import StableDiffusionPipeline
import torch
import gradio as gr
from PIL import Image
import uuid
import os

# Load Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe = pipe.to("cuda")

# Ensure image output folder exists
os.makedirs("generated_images", exist_ok=True)

# Image generation function
def generate_image(prompt):
    image = pipe(prompt).images[0]
    filename = f"generated_images/{uuid.uuid4().hex[:8]}.png"
    image.save(filename)
    return image, filename

In [ ]:
with gr.Blocks(css="""
.gradio-container {
    background: linear-gradient(135deg, #0f0c29, #302b63, #24243e);
    padding: 50px 20px;
    min-height: 100vh;
}

/* --- Targeting the h1 tag directly for reliability --- */
#main-title h1 {
    text-align: center;
    font-size: 2rem;
    font-weight: 900;
    margin-bottom: 50px;
    background: -webkit-linear-gradient(45deg, #ffb347, #ffd700);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
}

#promptbox input {
    font-size: 18px;
    padding: 12px;
    border: 2px solid #c800c8;
    border-radius: 10px;
}

#genbtn button {
    background: linear-gradient(45deg, #ff66cc, #cc00ff);
    color: white;
    font-weight: bold;
    font-size: 20px;
    padding: 14px 28px;
    border: none;
    border-radius: 12px;
    margin-top: 10px;
    transition: transform 0.2s ease, box-shadow 0.2s ease;
    box-shadow: 0 4px 12px rgba(0,0,0,0.2);
}
#genbtn button:hover {
    transform: scale(1.05);
    box-shadow: 0 6px 18px rgba(0,0,0,0.25);
}
""") as demo:

    # --- Wrapping the title in an h1 tag ---
    gr.Markdown("<h1>✨ AI Text-to-Image Generator</h1>", elem_id="main-title")

    prompt = gr.Textbox(placeholder="Describe your image...", label="Prompt", elem_id="promptbox")

    btn = gr.Button("🎨 Generate Image", elem_id="genbtn")

    image_output = gr.Image(label="Output Image", type="pil")
    file_output = gr.File(label="Download Image")

    btn.click(fn=run, inputs=prompt, outputs=[image_output, file_output])

demo.launch(share=True)